<a href="https://colab.research.google.com/github/tarumi283/tarumi/blob/main/Cosinor%20for%20fitting%20data_24well_version1.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install numpy scipy

In [2]:
from google.colab import files
files.upload()

print("アップロード完了。次のセルで INPUT_PATH を書いてください（ファイル名だけでOK）。")


Saving 2025-0826-1048_Plate1_F0_N_D.csv to 2025-0826-1048_Plate1_F0_N_D.csv
アップロード完了。次のセルで INPUT_PATH を書いてください（ファイル名だけでOK）。


In [3]:
from pathlib import Path

# ★アップロードしたファイル名（例）
INPUT_PATH = "/content/2025-0826-1048_Plate1_F0_N_D.csv"

# period scan settings
PERIOD_START = 20
PERIOD_END   = 30
PERIOD_STEP  = 0.1

# plate CSV settings
ANALYZE_ALL_WELLS = True   # True: 全well解析
WELL_NAME = "A1"           # ANALYZE_ALL_WELLS=Falseの時だけ有効
EXCLUDE_BACKGROUND = True

# output
OUTPUT_DIR = "cosinor_results"

# ---- check
p = Path(INPUT_PATH)
print("INPUT_PATH:", p)
print("Exists?:", p.exists())
if not p.exists():
    raise FileNotFoundError(
        "INPUT_PATH が見つかりません。アップロードしたファイル名と完全一致しているか確認してください。"
    )

USE_BINNING = True          # True: binningしてから解析 / False: 生データのまま解析
BIN_SIZE_H  = 0.1666667     # 10 min = 10/60 = 0.1666667 hours（例）
BIN_START   = 0             # 使い始めるbin番号（0始まり）
BIN_END     = 288          # 使い終わるbin番号（Noneなら最後まで）
# 例：最初の24時間を捨てたい(10分間隔)なら BIN_START = 24*6 = 144


INPUT_PATH: /content/2025-0826-1048_Plate1_F0_N_D.csv
Exists?: True


In [4]:
# ===== Cell 4: Cosinor core（EXE寄せ + acrophaseは“負表記” + PeakTimeも一貫） =====
# - モデル当てはめ： y = b0 + bc*cos(wt) + bs*sin(wt)
# - EXE寄せの位相定義： y = M + A*cos(wt - phi)
#     bc = A*cos(phi), bs = A*sin(phi), phi = atan2(bs, bc)
# - acrophase（表示用）は 0..360 を必ず負側で表示：phi_display = - (phi mod 360)（0は0）
# - PeakTime（実際のピーク時刻）は、内部の phi(0..360) を使って計算：
#     t_peak = (phi_mod360 / 360) * period
#   ※表示は負でも計算は正しいピーク位置になります

from __future__ import annotations

import math
from dataclasses import dataclass
from typing import List, Tuple

import numpy as np
from scipy.stats import f as f_dist


@dataclass
class PeriodRow:
    period: float
    f_stat: float
    p_value: float
    mesor: float
    amplitude: float
    acrophase_deg: float   # 表示用：0..360を負側にした値（例: -267）


def _fit_cosinor(t: np.ndarray, y: np.ndarray, period: float) -> Tuple[float, float, float, float, float]:
    """
    Fit:
      y = b0 + bc*cos(wt) + bs*sin(wt)
    Return: b0, bc, bs, SSE_full, SSE_null
    """
    w = 2.0 * np.pi / period
    X = np.column_stack([np.ones_like(t), np.cos(w * t), np.sin(w * t)])
    beta, *_ = np.linalg.lstsq(X, y, rcond=None)

    y_hat = X @ beta
    resid = y - y_hat
    sse_full = float(np.sum(resid ** 2))

    y_mean = float(np.mean(y))
    sse_null = float(np.sum((y - y_mean) ** 2))

    return float(beta[0]), float(beta[1]), float(beta[2]), sse_full, sse_null


def _f_test(sse_full: float, sse_null: float, n: int) -> Tuple[float, float]:
    """F-test for adding cos+sin (df_num=2) vs intercept only (df_den=n-3)."""
    if n <= 3:
        return float("nan"), float("nan")
    df_den = n - 3
    if sse_full <= 0:
        return float("inf"), 0.0

    f_stat = ((sse_null - sse_full) / 2.0) / (sse_full / df_den)
    if f_stat < 0:
        f_stat = 0.0
    p = float(f_dist.sf(f_stat, 2, df_den))
    return float(f_stat), p


def _phi_deg_mod360_from_bcbs(bc: float, bs: float) -> float:
    """
    EXE寄せ:
      y = M + A*cos(wt - phi)
      bc = A*cos(phi), bs = A*sin(phi)
      phi = atan2(bs, bc)  (radians)
    Return: phi in degrees wrapped into [0, 360).
    """
    phi = math.atan2(bs, bc)  # radians
    deg = (phi * 180.0 / math.pi) % 360.0
    return float(deg)


def _amp_acrophase_display_neg(bc: float, bs: float) -> Tuple[float, float]:
    """
    amplitude と “表示用acrophase（負表記）” を返す。
      phi_mod360 in [0,360)
      display = 0なら0、それ以外は -phi_mod360
    """
    amp = math.sqrt(bc * bc + bs * bs)
    phi_mod360 = _phi_deg_mod360_from_bcbs(bc, bs)
    if phi_mod360 == 0.0:
        display = 0.0
    else:
        display = -phi_mod360
    return float(amp), float(display)


def cosinor_periodogram_exe_style(
    t: np.ndarray,
    y: np.ndarray,
    period_start: float,
    period_end: float,
    period_step: float,
) -> Tuple[PeriodRow, List[PeriodRow]]:
    """Return (best_row, all_rows). best_row is the row with minimum p_value."""
    t = np.asarray(t, float)
    y = np.asarray(y, float)
    if t.shape != y.shape:
        raise ValueError("t and y must have the same shape.")
    if period_step <= 0:
        raise ValueError("period_step must be > 0")
    if period_end <= period_start:
        raise ValueError("period_end must be > period_start")

    periods = np.arange(period_start, period_end + 1e-12, period_step, dtype=float)
    n = int(t.size)

    rows: List[PeriodRow] = []
    for P in periods:
        b0, bc, bs, sse_full, sse_null = _fit_cosinor(t, y, float(P))
        f_stat, p = _f_test(sse_full, sse_null, n=n)
        amp, aph_display = _amp_acrophase_display_neg(bc, bs)

        rows.append(
            PeriodRow(
                period=float(P),
                f_stat=f_stat,
                p_value=p,
                mesor=b0,
                amplitude=amp,
                acrophase_deg=aph_display,  # 表示用（負）
            )
        )

    best = min(rows, key=lambda r: (r.p_value, -r.f_stat))
    return best, rows


def acrophase_to_peak_time(acrophase_deg: float, period_h: float) -> float:
    """
    PeakTime は「内部のphi(0..360)」で計算するのが正解。
    しかし acrophase_deg は“表示用の負表記”なので、まず phi_mod360 に戻す：

      if acrophase_deg == 0: phi_mod360=0
      else: phi_mod360 = (-acrophase_deg)  （例: -267 -> 267）

    そして EXE寄せの式：
      t_peak = (phi_mod360 / 360) * period
    を [0, period) に wrap して返す。
    """
    if acrophase_deg == 0.0:
        phi_mod360 = 0.0
    else:
        phi_mod360 = (-acrophase_deg) % 360.0

    t_peak = (phi_mod360 / 360.0) * period_h
    return float(t_peak % period_h)


In [5]:
import csv
import re
from typing import Dict, Tuple, Optional

import numpy as np
from pathlib import Path


def looks_like_plate_reader_csv(path: Path) -> bool:
    """Heuristic sniff for plate-reader CSV exports."""
    try:
        with path.open("r", encoding="utf-8", errors="ignore") as f:
            head = "".join([next(f) for _ in range(30)])
        return ("Time(h)" in head) and ("RLU" in head)
    except Exception:
        return False


def parse_plate_reader_csv(path: Path) -> Tuple[Dict[str, Tuple[np.ndarray, np.ndarray]], Dict[str, str]]:
    """
    Returns:
      series_by_well: {well: (t_hours, y_rlu)}
      meta: metadata
    """
    lines = path.read_text(encoding="utf-8", errors="ignore").splitlines()

    meta: Dict[str, str] = {}
    header_idx: Optional[int] = None

    # find header row and collect meta
    for i, ln in enumerate(lines[:200]):
        if ln.startswith('"Date","Time(h)"') or ln.startswith("Date,Time(h)") or ("Time(h)" in ln and "RLU" in ln and "Date" in ln):
            header_idx = i
            break
        m = re.match(r'^"([^"]+)"\s*,\s*(.*)$', ln)
        if m:
            meta[m.group(1).strip()] = m.group(2).strip().strip('"')

    if header_idx is None:
        raise ValueError("Could not find header row containing Date/Time(h)/RLU")

    # detect well-label row near header
    well_labels: list[str] = []
    well_pat = re.compile(r"^[A-H]\d+$")

    for back in range(max(0, header_idx - 12), header_idx):
        ln = lines[back]
        # require at least one typical well token
        if not any(w in ln for w in ("A1", "A2", "B1", "C1", "D1")):
            continue
        try:
            row = next(csv.reader([ln]))
        except Exception:
            continue
        cand = [c.strip().strip('"') for c in row if c.strip().strip('"')]
        if any(well_pat.fullmatch(c) for c in cand):
            well_labels = [c for c in cand if (c == "Background" or well_pat.fullmatch(c))]
            break

    # read from header row onward using csv.reader
    rows: list[list[str]] = []
    with path.open("r", encoding="utf-8", errors="ignore", newline="") as f:
        reader = csv.reader(f)
        for idx, row in enumerate(reader):
            if idx < header_idx:
                continue
            rows.append(row)

    header = rows[0]
    data_rows = rows[1:]

    def clean(s: str) -> str:
        return s.strip().strip('"')

    time_cols = [i for i, c in enumerate(header) if clean(c) == "Time(h)"]
    rlu_cols = [i for i, c in enumerate(header) if clean(c) == "RLU"]
    if not time_cols or not rlu_cols or len(time_cols) != len(rlu_cols):
        raise ValueError("Unexpected header: cannot find paired Time(h)/RLU columns.")

    pairs = list(zip(time_cols, rlu_cols))

    # assign names
    if well_labels and len(well_labels) >= len(pairs):
        names = well_labels[: len(pairs)]
    else:
        names = ["Background"] + [f"Well{i}" for i in range(1, len(pairs))]

    series_by_well: Dict[str, Tuple[np.ndarray, np.ndarray]] = {}
    for name, (ti, yi) in zip(names, pairs):
        t_vals: list[float] = []
        y_vals: list[float] = []
        for r in data_rows:
            if len(r) <= max(ti, yi):
                continue
            ts = r[ti].strip()
            ys = r[yi].strip()
            if ts == "" or ys == "":
                continue
            try:
                t_vals.append(float(ts))
                y_vals.append(float(ys))
            except ValueError:
                continue

        if len(t_vals) >= 4:
            series_by_well[name] = (np.asarray(t_vals, float), np.asarray(y_vals, float))

    if not series_by_well:
        raise ValueError("No valid series extracted from CSV.")
    return series_by_well, meta


In [12]:
# ===== Cell 6: binning（位相ズレの原因修正：bin中心ではなく実データ時刻を使う） =====

import numpy as np

def bin_time_series(t, y, bin_size_h: float, reducer: str = "mean", time_stamp: str = "mean"):
    """
    t: hours（float）
    y: signal
    bin_size_h: bin width (hours)
    reducer: "mean" or "median"
    time_stamp:
      - "mean" : bin内のt平均（おすすめ：実測時刻ベースで位相が合いやすい）
      - "first": bin内の最初のt（EXEが「最初の点」を基準にしてる場合に合いやすい）
      - "center": 旧仕様（bin中心） ※位相がズレやすいので非推奨

    Returns: (t_binned, y_binned)
    """
    t = np.asarray(t, float)
    y = np.asarray(y, float)
    if len(t) != len(y):
        raise ValueError("t and y length mismatch")
    if bin_size_h <= 0:
        raise ValueError("bin_size_h must be > 0")

    # sort by time
    idx = np.argsort(t)
    t = t[idx]
    y = y[idx]

    t0 = float(t[0])
    # bin index for each sample
    b = np.floor((t - t0) / bin_size_h).astype(int)

    nb = int(b.max()) + 1
    t_out = np.empty(nb, float)
    y_out = np.empty(nb, float)

    for k in range(nb):
        m = (b == k)
        if not np.any(m):
            t_out[k] = np.nan
            y_out[k] = np.nan
            continue

        tt = t[m]
        yy = y[m]

        # --- y aggregation ---
        if reducer == "median":
            y_out[k] = np.nanmedian(yy)
        else:
            y_out[k] = np.nanmean(yy)

        # --- time stamp (ここが重要) ---
        if time_stamp == "first":
            t_out[k] = float(tt[0])
        elif time_stamp == "center":
            t_out[k] = t0 + (k + 0.5) * bin_size_h
        else:  # "mean"
            t_out[k] = float(np.nanmean(tt))

    ok = ~np.isnan(t_out) & ~np.isnan(y_out)
    return t_out[ok], y_out[ok]


def select_bins(t_b, y_b, start_bin: int = 0, end_bin: int | None = None):
    """
    start_bin: 0-based inclusive
    end_bin: 0-based exclusive (Python slice). None means to the end.
    """
    if start_bin < 0:
        raise ValueError("BIN_START must be >= 0")
    n = len(y_b)
    if end_bin is None:
        end_bin = n
    if end_bin <= start_bin:
        raise ValueError("BIN_END must be > BIN_START (or None)")
    end_bin = min(end_bin, n)
    return t_b[start_bin:end_bin], y_b[start_bin:end_bin]



In [13]:
# ===== Cell 7: TXT report only (fixed) =====

import re
from pathlib import Path
from typing import List, Optional

def safe_filename(name: str) -> str:
    return re.sub(r"[^A-Za-z0-9_\-\.]+", "_", name.strip()) or "output"

def format_exe_report(
    best: PeriodRow,
    rows: List[PeriodRow],
    title: str,
    peak_time_h: float,
    meta: Optional[dict] = None
) -> str:
    out: List[str] = []
    out.append(title)
    if meta:
        for k in ("Plate format", "Date", "Time", "Integral time", "Interval time", "Measurement time", "Background"):
            if k in meta:
                out.append(f"{k}: {meta[k]}")
        out.append("")

    out.append("Cosinor (best period)")
    out.append(f"Best period {best.period:.6g} h")
    out.append(f"MESOR      {best.mesor:.6g}")
    out.append(f"Amplitude  {best.amplitude:.6g}")
    out.append(f"Acrophase  {best.acrophase_deg:.6g} deg")
    out.append(f"Peak time  {peak_time_h:.6g} h")
    out.append(f"F          {best.f_stat:.6g}")
    out.append(f"p          {best.p_value:.6g}")
    out.append("")
    out.append("Periodogram")
    out.append("Period\tF\tp")
    for r in rows:
        out.append(f"{r.period:.6g}\t{r.f_stat:.6g}\t{r.p_value:.6g}")
    return "\n".join(out) + "\n"


# ---- run
inp = Path(INPUT_PATH)
outdir = Path(OUTPUT_DIR)
outdir.mkdir(parents=True, exist_ok=True)

written = []

if inp.suffix.lower() == ".csv" and looks_like_plate_reader_csv(inp):
    series_by_well, meta = parse_plate_reader_csv(inp)

    if ANALYZE_ALL_WELLS:
        chosen = dict(series_by_well)
        if EXCLUDE_BACKGROUND:
            chosen.pop("Background", None)
    else:
        lut = {k.lower(): k for k in series_by_well.keys()}
        if WELL_NAME.lower() not in lut:
            raise KeyError(f"WELL_NAME '{WELL_NAME}' not found. Available: {', '.join(series_by_well.keys())}")
        key = lut[WELL_NAME.lower()]
        chosen = {key: series_by_well[key]}

    step = PERIOD_STEP  # ★ここが正しい（セル3で設定した値を使う）

    for well, (t, y) in chosen.items():
        if USE_BINNING:
            t2, y2 = bin_time_series(t, y, BIN_SIZE_H, reducer="mean")
            t2, y2 = select_bins(t2, y2, BIN_START, BIN_END)
        else:
            t2, y2 = t, y

        best, rows = cosinor_periodogram_exe_style(t2, y2, PERIOD_START, PERIOD_END, step)
        peak_time_h = acrophase_to_peak_time(best.acrophase_deg, best.period)

        title = f"Cosinor Periodogram Result  ({inp.name} / {well})"
        rep = format_exe_report(best, rows, title, peak_time_h, meta=meta)

        out_path = outdir / f"{safe_filename(well)}.txt"
        out_path.write_text(rep, encoding="utf-8")
        written.append(str(out_path))

else:
    raise ValueError("この版は plate reader CSV（Time(h)/RLU のワイドCSV）向けです。")

print("[OK] wrote:")
for p in written[:30]:
    print(" -", p)
if len(written) > 30:
    print(f" ... ({len(written)} files) total")
print("Output dir:", outdir.resolve())



[OK] wrote:
 - cosinor_results/A1.txt
 - cosinor_results/A2.txt
 - cosinor_results/A3.txt
 - cosinor_results/A4.txt
 - cosinor_results/A5.txt
 - cosinor_results/A6.txt
 - cosinor_results/B1.txt
 - cosinor_results/B2.txt
Output dir: /content/cosinor_results


In [14]:
# ===== Cell 8: 実行セル（窓基準の位相に修正済み・Excel出力も） =====

%pip -q install pandas openpyxl

import re
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Optional

# ---------- utils ----------
def safe_filename(name: str) -> str:
    return re.sub(r"[^A-Za-z0-9_\-\.]+", "_", str(name).strip()) or "output"

def format_exe_report(
    best: PeriodRow,
    rows: List[PeriodRow],
    title: str,
    peak_time_h: float,
    meta: Optional[dict] = None
) -> str:
    out: List[str] = []
    out.append(title)
    if meta:
        for k in ("Plate format", "Date", "Time", "Integral time", "Interval time", "Measurement time", "Background"):
            if k in meta:
                out.append(f"{k}: {meta[k]}")
        out.append("")

    out.append("Cosinor (best period)")
    out.append(f"Best period {best.period:.6g} h")
    out.append(f"MESOR      {best.mesor:.6g}")
    out.append(f"Amplitude  {best.amplitude:.6g}")
    out.append(f"Acrophase  {best.acrophase_deg:.6g} deg")
    out.append(f"Peak time  {peak_time_h:.6g} h")
    out.append(f"F          {best.f_stat:.6g}")
    out.append(f"p          {best.p_value:.6g}")
    out.append("")
    out.append("Periodogram")
    out.append("Period\tF\tp")
    for r in rows:
        out.append(f"{r.period:.6g}\t{r.f_stat:.6g}\t{r.p_value:.6g}")
    return "\n".join(out) + "\n"


# ---------- run ----------
inp = Path(INPUT_PATH)
outdir = Path(OUTPUT_DIR)
outdir.mkdir(parents=True, exist_ok=True)

# 前提チェック（不足ならセル実行順が原因）
required = [
    "looks_like_plate_reader_csv",
    "parse_plate_reader_csv",
    "bin_time_series",
    "select_bins",
    "cosinor_periodogram_exe_style",
    "acrophase_to_peak_time",
]
missing = [name for name in required if name not in globals()]
if missing:
    raise RuntimeError("前のセルが実行されていません。不足: " + ", ".join(missing))

# CSV 判定＆読み込み
if not (inp.suffix.lower() == ".csv" and looks_like_plate_reader_csv(inp)):
    raise ValueError("この版は plate reader CSV（Time(h)/RLU のワイドCSV）向けです。")

series_by_well, meta = parse_plate_reader_csv(inp)

print("=== Wells (first 30) ===")
print(list(series_by_well.keys())[:30])

# well選択
if ANALYZE_ALL_WELLS:
    chosen = dict(series_by_well)
    if EXCLUDE_BACKGROUND:
        chosen.pop("Background", None)
else:
    lut = {k.lower(): k for k in series_by_well.keys()}
    if WELL_NAME.lower() not in lut:
        raise KeyError(f"WELL_NAME '{WELL_NAME}' not found. Available: {', '.join(series_by_well.keys())}")
    key = lut[WELL_NAME.lower()]
    chosen = {key: series_by_well[key]}

# デバッグ対象well
DEBUG_WELL = "A1" if "A1" in chosen else list(chosen.keys())[0]
print("DEBUG_WELL =", DEBUG_WELL)

written_txt = []
summary_rows = []

step = PERIOD_STEP  # セル3の設定をそのまま使う

for well, (t, y) in chosen.items():
    t = np.asarray(t, float)
    y = np.asarray(y, float)

    # --- binning + 窓指定 ---
    if USE_BINNING:
        t2, y2 = bin_time_series(t, y, BIN_SIZE_H, reducer="mean", time_stamp="mean")
        t2, y2 = select_bins(t2, y2, BIN_START, BIN_END)
    else:
        t2, y2 = t, y

    # ===== ここが今回の修正ポイント（窓基準に揃える）=====
    # acrophase/peak time を「あなたが切った窓の先頭=0」で出す
    # period/amplitude は変わらない（位相だけが安定する）
    t2 = t2 - float(t2[0])
    # =======================================================

    # デバッグ（1 wellだけ）
    if well == DEBUG_WELL:
        print("\n=== DEBUG (after window + t0 shift) ===")
        print("well:", well)
        print("used n =", len(t2), "t2_0 =", float(t2[0]), "t2_end =", float(t2[-1]),
              "y2_mean =", float(np.mean(y2)))
        print("t2 first 10:", np.round(t2[:10], 6))
        print("y2 first 10:", np.round(y2[:10], 6))
        print("=== DEBUG END ===\n")

    # --- cosinor scan ---
    best, rows = cosinor_periodogram_exe_style(t2, y2, PERIOD_START, PERIOD_END, step)

    # --- peak time ---
    peak_time_h = acrophase_to_peak_time(best.acrophase_deg, best.period)

    # --- txt report ---
    title = f"Cosinor Periodogram Result  ({inp.name} / {well})"
    rep = format_exe_report(best, rows, title, peak_time_h, meta=meta)
    out_path = outdir / f"{safe_filename(well)}.txt"
    out_path.write_text(rep, encoding="utf-8")
    written_txt.append(str(out_path))

    # --- summary row ---
    summary_rows.append({
        "Well": well,
        "BestPeriod_h": best.period,
        "MESOR": best.mesor,
        "Amplitude": best.amplitude,
        "Acrophase_deg": best.acrophase_deg,
        "PeakTime_h": peak_time_h,
        "F": best.f_stat,
        "p": best.p_value,
    })

# ---- write summary to Excel/CSV
df = pd.DataFrame(summary_rows)

# Wellが A1..H12 形式のときだけ整列
m_row = df["Well"].astype(str).str.extract(r"^([A-H])")[0]
m_col = df["Well"].astype(str).str.extract(r"(\d+)$")[0]
if m_row.notna().all() and m_col.notna().all():
    df["__row"] = m_row
    df["__col"] = m_col.astype(int)
    df = df.sort_values(["__row", "__col"]).drop(columns=["__row", "__col"])

xlsx_path = outdir / "cosinor_summary.xlsx"
csv_path  = outdir / "cosinor_summary.csv"
df.to_excel(xlsx_path, index=False)
df.to_csv(csv_path, index=False)

print("[OK] wrote txt:")
for p in written_txt[:20]:
    print(" -", p)
if len(written_txt) > 20:
    print(f" ... ({len(written_txt)} files) total")

print("\n[OK] wrote summary:")
print(" -", xlsx_path)
print(" -", csv_path)

df


=== Wells (first 30) ===
['Background', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'B1', 'B2']
DEBUG_WELL = A1

=== DEBUG (after window + t0 shift) ===
well: A1
used n = 288 t2_0 = 0.0 t2_end = 49.0 y2_mean = 148.15625
t2 first 10: [0.       0.166667 0.416667 0.666667 0.916667 1.166667 1.416667 1.666667
 1.916667 2.166667]
y2 first 10: [1265.  1322.  1395.5 1481.  1536.  1627.  1804.  1956.  2131.  2228. ]
=== DEBUG END ===

[OK] wrote txt:
 - cosinor_results/A1.txt
 - cosinor_results/A2.txt
 - cosinor_results/A3.txt
 - cosinor_results/A4.txt
 - cosinor_results/A5.txt
 - cosinor_results/A6.txt
 - cosinor_results/B1.txt
 - cosinor_results/B2.txt

[OK] wrote summary:
 - cosinor_results/cosinor_summary.xlsx
 - cosinor_results/cosinor_summary.csv


,Well,BestPeriod_h,MESOR,Amplitude,Acrophase_deg,PeakTime_h,F,p
0,A1,24.7,180.394941,2137.660415,-79.938200,5.484649,1097.198359,1.321552e-134
1,A2,24.6,187.389105,2248.297637,-81.759252,5.586882,1052.914120,2.356399e-132
2,A3,24.5,256.019119,2774.810734,-76.083718,5.177920,1017.260673,1.762481e-130
3,A4,24.4,266.305399,2715.677168,-75.923622,5.145934,1124.970727,5.622921e-136
4,A5,24.8,256.901965,2429.768971,-76.131482,5.244613,1036.633988,1.662908e-131
5,A6,24.8,249.827418,2412.535937,-83.061435,5.722010,1086.080833,4.770741e-134
6,B1,24.9,159.781492,1986.827529,-75.238799,5.204017,2113.418818,1.174963e-171
7,B2,24.7,160.885581,1871.788783,-78.066509,5.356230,1926.785357,2.595454e-166


In [11]:

from google.colab import files

files.download(f"{OUTPUT_DIR}/cosinor_summary.xlsx")
# ついでにCSVも欲しければ:
# files.download(f"{OUTPUT_DIR}/cosinor_summary.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>